In [26]:
function isRecursive(expr :: Expr) :: Bool
    return true #TODO
end

function isTailRecursive(expr :: Expr) :: Bool
    return true #TODO
end

function isFunctionDefinition(expr :: Expr) :: Bool
    return isa(expr, Expr) && expr.head === :function
end

macro tco(f)
    if !isFunctionDefinition(f)
        @error "Not a function definiion!"
        return f
    end
    
    if !isTailRecursive(f)
        @error "Not a tail recursive function!"
        return f
    end
    
    name = f.args[1]
    body = f.args[2]
    
    allButLast = body.args[1:end-1]
    last = body.args[end]
    
    left = [:($(i.args[1])) for i in name.args[2:end]]
    right = [i for i in last.args[1].args[2:end]]
    assignment = :( $(Expr(:tuple, left...)) = $(Expr(:tuple, right...)) )
    
    newBody = [allButLast; assignment]
    
    if name.args[1] != last.args[1].args[1]
        @error "Not a tail recursive function!"
        return f
    end
    
    return Expr(
        f.head,
        f.args[1],
        Expr(:block,
            Expr(
                :while,
                true,
                Expr(
                    body.head,
                    newBody...))))
    
end

@tco (macro with 1 method)

In [27]:
@macroexpand @tco function fib(n :: Int64, a :: Int64, b :: Int64)
    if (n == 1)
        return a
    end
    
    if (n == 2)
        return b
    end
    
    return fib(n-1, b, a+b)
end

:(function (Main.fib)(#167#n::Main.Int64, #168#a::Main.Int64, #169#b::Main.Int64)
      while true
          #= In[27]:2 =#
          if #167#n == 1
              #= In[27]:3 =#
              return #168#a
          end
          #= In[27]:6 =#
          if #167#n == 2
              #= In[27]:7 =#
              return #169#b
          end
          #= In[27]:10 =#
          (#167#n, #168#a, #169#b) = (#167#n - 1, #169#b, #168#a + #169#b)
      end
  end)

In [28]:
@tco function fib_with_tco(n :: Int64, a :: Int64, b :: Int64)
    if (n == 1)
        return a
    end
    
    if (n == 2)
        return b
    end
    
    return fib_with_tco(n-1, b, a+b)
end

fib_with_tco (generic function with 2 methods)

In [29]:
function fib_without_tco(n :: Int64, a :: Int64, b :: Int64)
    if (n == 1)
        return a
    end
    
    if (n == 2)
        return b
    end
    
    return fib_without_tco(n-1, b, a+b)
end

fib_without_tco (generic function with 2 methods)

In [30]:
using BenchmarkTools

In [56]:
@time fib_with_tco(1000000, 1, 1)

  0.000628 seconds (5 allocations: 176 bytes)


-4249520595888827205

In [57]:
@time fib_without_tco(1000000, 1, 1)

StackOverflowError: StackOverflowError:

In [58]:
@code_lowered fib_with_tco(10, 1, 1)

CodeInfo(
1 ─       #171#a = a
│         #170#n = n
└──       #172#b = b
2 ┄       goto #8 if not true
3 ─ %5  = #170#n == 1
└──       goto #5 if not %5
4 ─       return #171#a
5 ─ %8  = #170#n == 2
└──       goto #7 if not %8
6 ─       return #172#b
7 ─ %11 = #170#n - 1
│   %12 = #172#b
│   %13 = #171#a + #172#b
│         #170#n = %11
│         #171#a = %12
│         #172#b = %13
└──       goto #2
8 ─       return
)

In [59]:
@code_lowered fib_without_tco(10, 1, 1)

CodeInfo(
1 ─ %1 = n == 1
└──      goto #3 if not %1
2 ─      return a
3 ─ %4 = n == 2
└──      goto #5 if not %4
4 ─      return b
5 ─ %7 = n - 1
│   %8 = a + b
│   %9 = (Main.fib_without_tco)(%7, b, %8)
└──      return %9
)

In [76]:
@tco function factorial_with_tco(n::Int64, a::BigInt)
    
    if (n == 1)
        return a
    end
    
    return factorial_with_tco(n-1, n*a)
end

factorial_with_tco (generic function with 2 methods)

In [77]:
function factorial_without_tco(n::Int64, a::BigInt)
    
    if (n == 1)
        return a
    end
    
    return factorial_without_tco(n-1, n*a)
end

factorial_without_tco (generic function with 2 methods)

In [85]:
factorial_with_tco(100, BigInt(1))

  0.000066 seconds (303 allocations: 8.000 KiB)


93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

In [86]:
factorial_without_tco(100, BigInt(1))

  0.000044 seconds (303 allocations: 8.000 KiB)


93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000